# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [9]:
from dotenv import load_dotenv
import os
load_dotenv()
price_data_dir = os.getenv('PRICE_DATA')
print("PRICE_DATA", price_data_dir)

PRICE_DATA ../../05_src/data/prices/


In [10]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [11]:
import os
from glob import glob

parquet_files = glob(os.path.join(price_data_dir, "**/*.parquet/*.parquet"))
dd_px=dd.read_parquet(parquet_files).set_index("ticker")
print(dd_px.columns)

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'sector',
       'subsector', 'year'],
      dtype='object')


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [12]:
import numpy as np
from dask.dataframe import rolling

dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag = x['Close'].shift(1),
                       Adj_close_lag = x['Adj Close'].shift(1))
).assign(
    returns = lambda x: x['Adj Close']/x['Adj_close_lag'] - 1
).assign(
    hi_to_range = lambda x: x['High'] -x ['Low']
))

C:\Users\daoda\AppData\Local\Temp\ipykernel_31392\1877060989.py:4: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [13]:
pdf = dd_feat.compute()
print(pdf.head())

pdf['rolling_avg_return'] = pdf['returns'].rolling(10).mean()
print(pdf.head())

             Date       Open       High        Low      Close  Adj Close  \
ticker                                                                     
OXY    2004-01-02  20.191145  20.382757  20.095339  20.138451  11.842054   
OXY    2004-01-05  20.358805  20.933641  20.334854  20.933641  12.309657   
OXY    2004-01-06  20.933641  20.933641  20.612692  20.722870  12.185718   
OXY    2004-01-07  20.722870  20.722870  20.286951  20.492933  12.050504   
OXY    2004-01-08  20.670176  20.684546  20.502514  20.651014  12.143458   

         Volume  sector                           subsector  year  Close_lag  \
ticker                                                                         
OXY     4034814  Energy  Oil & Gas Exploration & Production  2004        NaN   
OXY     4055690  Energy  Oil & Gas Exploration & Production  2004  20.138451   
OXY     3941501  Energy  Oil & Gas Exploration & Production  2004  20.933641   
OXY     2708803  Energy  Oil & Gas Exploration & Production  2004  

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

No, it was not necessary to convert to pandas to calculate the moving average return. Dask has support for rolling window operations which can be used to calculate moving averages.
Yes, it is better to do it in Dask, because Dask is designed to handle larger-than-memory datasets by parallelizing operations and distributing computation. Using Dask for rolling operations would allow us to take advantage of this parallelism and potentially reduce memory usage and computation time.

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.